In [ ]:
%load_ext rpy2.ipython

In [3]:
%%R
lapply(c("dplyr","Seurat","HGNChelper","openxlsx"), library, character.only = T)

load('../data/reference.RData')

sample_meta<-'AD_HS_00001	Molecular characterization of selectively vulnerable neurons in Alzheimer’s Disease	20	H.Sapiens	AD	Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Superior frontal gyrus;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex;Entorhinal cortex	Braak:0;0;0;2;2;2;2;6;6;6;0;0;0;2;2;2;2;6;6;6	ExcitatoryNeuron;InhibitoryNeuron;Oligodendrocyte;OPC;Astrocyte;Microglia;Endothelia;Pericyte	GSE147528	Single-nucleus RNA-seq	10x Genomics	Male	60;50;71;72;87;91;77;82;72;82;60;50;71;72;87;91;77;82;72;82	SRR11422700;SRR11422701;SRR11422702;SRR11422703;SRR11422704;SRR11422705;SRR11422706;SRR11422707;SRR11422708;SRR11422709;SRR11422710;SRR11422711;SRR11422712;SRR11422713;SRR11422714;SRR11422715;SRR11422716;SRR11422717;SRR11422718;SRR11422719'
split_meta<-unlist(strsplit(sample_meta, '\t'))
samples<-unlist(strsplit(split_meta[14],';'))
disease<-rep(split_meta[5],length(samples))
stage<-unlist(strsplit(unlist(strsplit(split_meta[7],':'))[2],';'))
gender<-rep(split_meta[12],length(samples))
age<-unlist(strsplit(split_meta[13],';'))

sample_meta<-data.frame(samples=samples,disease=disease,stage=stage,gender=gender,age=age)
sample_meta[sample_meta$stage=='0','disease']<-'Control'
rownames(sample_meta)<-sample_meta$samples

In [100]:
%%R
library(stringr)
# args <- commandArgs(trailingOnly=TRUE)
args<-c('../data/3/scte','../data/3/cell_umap.txt','SRR11422700','/home/wdeng3/workspace/software/sc-type','../../universal_data/rmsk/rmsk_GRCh38.txt')
data_path<-args[1]
out_path<-args[2]
sample_tag<-args[3]
sctype_path<-args[4]
rmsk_path<-args[5]
rmsk<-read.table(rmsk_path,sep='\t',header=F)
to_remove<-unlist(rmsk[!(rmsk$V12 %in% c('LINE','SINE','LTR')),'V11'])
to_remove<-str_replace_all(to_remove,'-','\\.')

reps<-unlist(rmsk[rmsk$V12 %in% c('LINE','SINE','LTR'),'V11'])
reps<-str_replace_all(reps,'-','\\.')

create_seurat<-function(sample_tag){
    ## load data
    scte.data <- t(read.csv(paste0(data_path,'/',sample_tag,'/',sample_tag,'.csv'),check.names=F, row.names = 1))
    # remove repeats from gene expression matrix

    scte.gene<-scte.data[!(row.names(scte.data) %in% to_remove),]
    # normalize and scale data
    scte <- CreateSeuratObject(counts = scte.gene, project = sample_tag, min.cells = 3, min.features = 200)
    # normalize data
    scte[["percent.mt"]] <- PercentageFeatureSet(scte, pattern = "^MT.|^MT-")
    scte <- subset(scte, subset = nFeature_RNA > 200 & nFeature_RNA < 2500 & percent.mt < 5)
    scte <- SCTransform(scte, vars.to.regress = "percent.mt", verbose = FALSE)

    cell_meta<-scte@meta.data
    cell_meta$disease<-sample_meta[cell_meta$orig.ident,'disease']

    cell_meta$stage<-sample_meta[cell_meta$orig.ident,'stage']
    cell_meta$gender<-sample_meta[cell_meta$orig.ident,'gender']
    cell_meta$age<-sample_meta[cell_meta$orig.ident,'age']
    scte@meta.data<-cell_meta
    scte
}

scte<-create_seurat(sample_tag)

scte <- FindVariableFeatures(scte, selection.method = "vst", nfeatures = 2000)
# scale and run PCA
scte <- ScaleData(scte, features = rownames(scte))
scte <- RunPCA(scte, features = VariableFeatures(object = scte))

sdv<-Stdev(scte,reduction='pca')
sdv<-sdv[sdv>2]
npca<-length(sdv)

scte <- FindNeighbors(scte, dims = 1:npca)
scte <- FindClusters(scte, resolution = 0.8)
scte <- RunUMAP(scte, dims = 1:npca)


scte.anchors <- FindTransferAnchors(reference = reference, query = scte,
    dims = 1:30, reference.reduction = "pca")
scte.query <- MapQuery(anchorset = scte.anchors, reference = reference, query = scte,
    refdata = list(celltype = "broad.cell.type"), reference.reduction = "pca", reduction.model = "umap")
cell_meta<-scte.query@meta.data

cell_meta$UMAP_1<-scte.query@reductions[['umap']]@cell.embeddings[,1]
cell_meta$UMAP_2<-scte.query@reductions[['umap']]@cell.embeddings[,2]
scte.query@meta.data<-cell_meta
write.table(cell_meta,out_path,sep='\t',quote=F,row.names=T)

## get gene expression from scte_query
scte.query.gene<-as.data.frame(t(as.data.frame(GetAssayData(scte.query, slot = "data"))))
scte.query.gene$UMAP_1<-scte.query@meta.data$UMAP_1
scte.query.gene$UMAP_2<-scte.query@meta.data$UMAP_2
write.table(scte.query.gene,'../data/3/cell_exp.txt',sep='\t',quote=F,row.names=T)

R[write to console]: Warning:
R[write to console]:  Feature names cannot have underscores ('_'), replacing with dashes ('-')

R[write to console]: Centering and scaling data matrix

  |                                                                            
  |                                                                      |   0%
  |                                                                            
  |===                                                                   |   4%
  |                                                                            
  |=====                                                                 |   7%
  |                                                                            
  |========                                                              |  11%
  |                                                                            
  |==========                                                            |  15%
  |               

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 6376
Number of edges: 212301

Running Louvain algorithm...


R[write to console]: 0%   10   20   30   40   50   60   70   80   90   100%

R[write to console]: [----|----|----|----|----|----|----|----|----|----|

R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: 

Maximum modularity in 10 random starts: 0.8550
Number of communities: 15
Elapsed time: 1 seconds


R[write to console]: 03:39:16 UMAP embedding parameters a = 0.9922 b = 1.112

R[write to console]: 03:39:17 Read 6376 rows and found 10 numeric columns

R[write to console]: 03:39:17 Using Annoy for neighbor search, n_neighbors = 30

R[write to console]: 03:39:17 Building Annoy index with metric = cosine, n_trees = 50

R[write to console]: 0%   10   20   30   40   50   60   70   80   90   100%

R[write to console]: [----|----|----|----|----|----|----|----|----|----|

R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *

  |                                                  | 0 % ~calculating  

R[write to console]: 
Integrating dataset 2 with reference dataset

R[write to console]: Finding integration vectors

R[write to console]: Integrating data



  |++++++++++++++++++++++++++++++++++++++++++++++++++| 100% elapsed=01s  


R[write to console]: Computing nearest neighbors

R[write to console]: Running UMAP projection

R[write to console]: 03:43:33 Read 6376 rows

R[write to console]: 03:43:33 Processing block 1 of 1

R[write to console]: 03:43:33 Commencing smooth kNN distance calibration using 1 thread
R[write to console]:  with target n_neighbors = 30

R[write to console]: 03:43:33 Initializing by weighted average of neighbor coordinates using 1 thread

R[write to console]: 03:43:33 Commencing optimization for 67 epochs, with 191280 positive edges

R[write to console]: Using method '
R[write to console]: umap
R[write to console]: '
R[write to console]: 

R[write to console]: 0%   10   20   30   40   50   60   70   80   90   100%

R[write to console]: [----|----|----|----|----|----|----|----|----|----|

R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]:

In [102]:
%%R
## get gene expression from scte_query
# scte.query.gene<-as.data.frame(t(as.data.frame(GetAssayData(scte.query, slot = "data"))))
# scte.query.gene$UMAP_1<-scte.query@meta.data$UMAP_1
# scte.query.gene$UMAP_2<-scte.query@meta.data$UMAP_2
write.table(scte.query.gene,'../data/3/cell_exp.txt',sep='\t',quote=F,row.names=T)

In [84]:
%%R


In [117]:
# import sys
# import math
# import pandas as pd

# sys.argv=['th','../data/3/cell_exp.txt','../www/mysql']
# out_path=sys.argv[2]
# cell_exp=pd.read_csv(sys.argv[1],sep='\t',index_col=0)

# umap=cell_exp[['UMAP_1','UMAP_2']]
# cell_exp=cell_exp.drop(['UMAP_1','UMAP_2'],axis=1)

exp_tables=[open(f'{out_path}/cell_exp_{i}.sql','w') for i in range(math.ceil((len(cell_exp.columns))/4000.0))]
gene_dict=open(f'{out_path}/gene_dict.sql','w')
gene_dict.write('''CREATE DATABASE IF NOT EXISTS scARE;
USE scARE;
DROP TABLE IF EXISTS GENE_DICT;
CREATE TABLE GENE_DICT (
    ID INT NOT NULL AUTO_INCREMENT,
    GENE varchar(255) NOT NULL,
    TABLE_ID INT NOT NULL,
    PRIMARY KEY (ID) );\n''')

224

In [118]:
header=cell_exp.columns
for i in range(len(exp_tables)):
    exp_tables[i].write('''CREATE DATABASE IF NOT EXISTS scARE;
USE scARE;
DROP TABLE IF EXISTS CELL_EXP_{i};
CREATE TABLE CELL_EXP_{i} (
ID INT NOT NULL AUTO_INCREMENT,
CELL varchar(255) NOT NULL,\n'''.format(i=i))

for i in range(len(cell_exp.columns)):
    index=math.floor(i/4000.0)
    exp_tables[index].write(f'`{header[i]}` FLOAT NOT NULL,\n')
    gene_dict.write(f'INSERT INTO GENE_DICT values({i},{header[i]},{index});\n')

for i in range(len(exp_tables)):
    exp_tables[i].write('UMAP_1 FLOAT NOT NULL,\n')
    exp_tables[i].write('UMAP_2 FLOAT NOT NULL,\n')
    exp_tables[i].write('PRIMARY KEY (ID) );\n')


import tqdm

In [119]:
import tqdm

for cell in tqdm.tqdm(cell_exp.index):
    for i in range(len(exp_tables)):
        values=','.join([str(i) for i in cell_exp.loc[cell][i*4000:(i+1)*4000]])
        values+=','+str(umap.loc[cell][0])+','+str(umap.loc[cell][1])
        exp_tables[i].write(f'INSERT INTO CELL_EXP_{i} values({cell},{values});\n')

for i in range(len(exp_tables)):
    exp_tables[i].close()
gene_dict.close()

100%|██████████| 6376/6376 [01:50<00:00, 57.55it/s]


In [ ]:
# ## assign cell type according to ScType tuturial

## load sc type and database
# source(paste0(sctype_path,"/R/gene_sets_prepare.R"))
# source(paste0(sctype_path,"/R/sctype_score_.R"))
# gs_list = gene_sets_prepare(paste0(sctype_path,'/ScTypeDB_full.xlsx'), "Brain" )
# # get cell-type by cell matrix
# es.max = sctype_score(scRNAseqData = scte[["RNA"]]@scale.data, scaled = TRUE, 
#                       gs = gs_list$gs_positive, gs2 = gs_list$gs_negative) 

# # merge by cluster
# cL_resutls = do.call("rbind", lapply(unique(scte@meta.data$seurat_clusters), function(cl){
#     es.max.cl = sort(rowSums(es.max[ ,rownames(scte@meta.data[scte@meta.data$seurat_clusters==cl, ])]), decreasing = !0)
#     head(data.frame(cluster = cl, type = names(es.max.cl), scores = es.max.cl, ncells = sum(scte@meta.data$seurat_clusters==cl)), 10)
# }))
# sctype_scores = cL_resutls %>% group_by(cluster) %>% top_n(n = 1, wt = scores)  

# # set low-confident (low ScType score) clusters to "unknown"
# sctype_scores$type[as.numeric(as.character(sctype_scores$scores)) < sctype_scores$ncells/4] = "Unknown"


# scte@meta.data$customclassif = ""
# for(j in unique(sctype_scores$cluster)){
#   cl_type = sctype_scores[sctype_scores$cluster==j,]; 
#   scte@meta.data$customclassif[scte@meta.data$seurat_clusters == j] = as.character(cl_type$type[1])
# }

# ## save cell type
# cell_types<-scte[['customclassif']]
